# Prepare a small reference genome

We cut out a small ref genome from the CHM13 ref genome and normalize all N's by replacing them by random DNA letters.

Since NanoSim itself does not scale well due to locking, we sometimes run it in parallel. 
NanoSim automatically replaces N's by random letters. To ensure the same reference across all parallel instances, we replace the N letters.


In [1]:
%load_ext autoreload
%autoreload 2

%cd ~/ont_project_all/ont_project
%pwd
from pathlib import Path

import gzip
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import bgzf
import pysam
import itertools

from simreaduntil.shared_utils.nanosim_parsing import case_convert_dna
from simreaduntil.shared_utils.utils import tqdm_with_name

/Users/maximilianmordig/ont_project_all/ont_project


In [2]:
import os
os.environ["PATH"] = str(Path("~/ont_project_all/tools/bin").expanduser()) + ":" + os.environ["PATH"]
!which minimap2

/Users/maximilianmordig/ont_project_all/tools/bin/minimap2


In [ ]:
run_dir = Path("runs/enrich_usecase")
orig_ref_genome_path = run_dir / "data/chm13v2.0.fa.gz"
ref_genome_path = run_dir / "data/chm13v2.0_normalized1000000firsttwo.fa.gz"

run_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
# use either the aws cli or curl to download the reference genome
!curl https://s3-us-west-2.amazonaws.com/human-pangenomics/T2T/CHM13/assemblies/analysis_set/chm13v2.0.fa.gz --output {orig_ref_genome_path}
# or
# !aws s3 --no-sign-request cp s3://human-pangenomics/T2T/CHM13/assemblies/analysis_set/chm13v2.0.fa.gz {orig_ref_genome_path}

In [ ]:
def normalize_fasta_modified(fasta_gz):
    # use SeqIO rather than pysam.FastaFile because it discards the additional chromosome name
    with gzip.open(fasta_gz, mode="rt") as f:
        for (i, record) in enumerate(SeqIO.parse(f, "fasta")):
            # yield (record.id, SeqIO.SeqRecord(seq=Seq(case_convert_dna(record.seq)), id=record.id, description=f"{record.description} normalized"))
            yield (record.id, SeqIO.SeqRecord(seq=Seq(case_convert_dna(record.seq[1_000_000:2_000_000])), id=record.id, description=f"{record.description} normalized"))
            if i>=1:
                break
            # print(record.format("fasta"))
        
# bgzip compression needed for pysam
with bgzf.open(ref_genome_path, "wt") as f:
    SeqIO.write(tqdm_with_name(normalize_fasta_modified(orig_ref_genome_path)), f, "fasta")
print(f"Written to '{ref_genome_path}'")

print(f"Chromosome names: {pysam.FastaFile(ref_genome_path).references}")

In [ ]:
# # take first chromosome from fasta file and write to new file
# with bgzf.open(enrich_ref_genome_path, "wt") as f:
#     with gzip.open(ref_genome_path, mode="rt") as f_in:
#         SeqIO.write(itertools.islice(SeqIO.parse(f_in, "fasta"), 1), f, "fasta")
# print(f"Chromosome names: {pysam.FastaFile(enrich_ref_genome_path).references}")

In [ ]:
%%bash -e

# cd '{run_dir}' # not working
cd runs/enrich_usecase
export PATH=~/ont_project_all/tools/bin:$PATH && which minimap2 # make minimap2 available
# minimap2 -d data/chm13v2.0_normalized1000000first.mmi data/chm13v2.0_normalized1000000first.fa.gz
minimap2 -d data/chm13v2.0_normalized1000000firsttwo.mmi data/chm13v2.0_normalized1000000firsttwo.fa.gz

## Enrich full genome

In [3]:
run_dir = Path("runs/enrich_usecase")
orig_ref_genome_path = run_dir / "data/chm13v2.0.fa.gz"
ref_genome_path = run_dir / "data/chm13v2.0_normalized.fa.gz"

run_dir.mkdir(exist_ok=True, parents=True)

In [5]:
# from simreaduntil.usecase_helpers.utils import normalize_fasta
# normalize_fasta(orig_ref_genome_path, ref_genome_path)

!normalize_fasta --help
!normalize_fasta {orig_ref_genome_path} {ref_genome_path}

# # todo: can be parallelized
# with bgzf.open(ref_genome_path, "wt") as f:
#     SeqIO.write(tqdm_with_name(normalize_fasta_gen(orig_ref_genome_path)), f, "fasta")
# print(f"Written to '{ref_genome_path}'")

# print(f"Chromosome names: {pysam.FastaFile(ref_genome_path).references}")

usage: normalize_fasta [-h] in_fasta_path out_fasta_path

Normalize a FASTA file to upper case and replacing N letters

positional arguments:
  in_fasta_path   input FASTA file
  out_fasta_path  output FASTA file

options:
  -h, --help      show this help message and exit
2023-08-07 09:58:39,755 - in_fasta_path=runs/enrich_usecase/data/chm13v2.0.fa.gz --- utils.py:145 (print_args) INFO ##
2023-08-07 09:58:39,755 - out_fasta_path=runs/enrich_usecase/data/chm13v2.0_normalized1.fa.gz --- utils.py:145 (print_args) INFO ##
2023-08-07 09:58:39,755 - Detected .gz extension for file 'runs/enrich_usecase/data/chm13v2.0_normalized1.fa.gz' --- utils.py:107 (get_fasta_open_method) INFO ##
0it [00:00, ?it/s]2023-08-07 09:58:39,788 - Detected .gz extension for file 'runs/enrich_usecase/data/chm13v2.0.fa.gz' --- utils.py:107 (get_fasta_open_method) INFO ##
^C
